In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd

In [13]:
# read in datasets
restInspections = pd.read_csv('raw_data\DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
openRest = pd.read_csv('raw_data\Open_Restaurant_Applications.csv')
rats_df = pd.read_csv('C:\\Users\\abonc\\OneDrive\\CornellTech\\Classes\\INFO 5268 PAML\\Project\\311_Rodent_Complaints.csv')
zipcodes = gpd.read_file('raw_data\\ZIP_CODE.zip')


C:\Users\abonc\AppData\Local\Temp\ipykernel_38132\906343043.py:4: DtypeWarning: Columns (17,33) have mixed types. Specify dtype option on import or set low_memory=False.
  rats_df = pd.read_csv('C:\\Users\\abonc\\OneDrive\\CornellTech\\Classes\\INFO 5268 PAML\\Project\\311_Rodent_Complaints.csv')


### Restaurant Inspection Dataset Data Cleaning

There are 202,467 rows in the dataset, relating to the number of inspections performed. However, each row is not a unique restaurant. Restaurants are supposed to be inspected each year, and will be reinspected if they have violations. We want only one data point for each restaurant for our analysis, specifically for 2022. This will be based on the "CAMIS" column, as that is a unique identifier for each restauruants (some restaurants might have the same name)

#### Initial Cleaning / Processing of dataset
Initial cleaning of dataset consists of removing irreleviant features and filtering dataset for rows with missing data. Additionally, filtering for analysis year of 2022. Each restaurant is supposed to be inspected each year. Since we are looking at rat complaints from 2022, we want to look at the restaurant inspection results from that time period. If a restaurant was inspected more than once during that year (typically done if there were critical violations that needed to be fixed), we will take the average score for that restaurant

In [14]:
# convert inspection data, grade data, and record data to datetime
restInspections['INSPECTION DATE'] = pd.to_datetime(restInspections['INSPECTION DATE'])
restInspections['GRADE DATE'] = pd.to_datetime(restInspections['GRADE DATE'])
restInspections['RECORD DATE'] = pd.to_datetime(restInspections['RECORD DATE'])

# filter out rows with no inspection grade
restInspections = restInspections[restInspections['GRADE'].notna()]

# filter out data that does not have lat / long in nyc
restInspections = restInspections[restInspections['Latitude']>35]
restInspections = restInspections[restInspections['Longitude']<-65]

# filter for 2022 inspections only, since that is the year we are analyzing for rat complaints
start_year = '2022-01-01'
end_year = '2022-12-31'
restInspections = restInspections[(restInspections['INSPECTION DATE']>=start_year) & (restInspections['INSPECTION DATE']<=end_year)]

Critical flags, establishment closed, and cuisine have been identified as features we want to explore as they related to our analysis. We will encode critical flag column and action column to bianary variables - 1 if there is a critical flag or establishment was closed and 0 if not. Cuisine will be one-hot encoded.

In [15]:
# critical flags are coded as 1, others are coded as 0
restInspections['CRITICAL FLAG'] = restInspections['CRITICAL FLAG'].map({'Critical':1, 'Not Critical':0, 'Not Applicable' :0})
# closing the restaurant actions are coded as 1, others are codded as 0
restInspections['ACTION'] = restInspections['ACTION'].map({'Violations were cited in the following area(s).': 0,
                                                           'Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.':1,
                                                           'Establishment re-opened by DOHMH.':0,
                                                           'No violations were recorded at the time of this inspection.': 0,
                                                           'Establishment re-closed by DOHMH.':1})

Some restaurants have multiple rows for 2022. Additionally, sometimes they have multiple rows on the same inspection data; in those cases, the score given is the same, but the violation description and critical flag awarded can be different. We processes the data to get one row for each restaurant that represents the average score given and the average number of critical flags given across all inspection days for that year.

In [16]:
# group by each restaurant and each inspection date, find the number of critical flags for each date
restInspections_critFlag = restInspections.groupby(by=['CAMIS', 'INSPECTION DATE']).sum().reset_index()
restInspections_critFlag = restInspections_critFlag[['CAMIS', 'INSPECTION DATE', 'CRITICAL FLAG', 'ACTION']]

# group by each restaurant and each inspection date to get the score for each inspection date
restInspections_score = restInspections.groupby(by=['CAMIS', 'INSPECTION DATE']).mean().reset_index()
restInspections_score = restInspections_score[['CAMIS', 'INSPECTION DATE', 'SCORE']]

# join two datasets together to get total number of critical flags and the score for each inspection date (will now have one row for each inspection date with total critical flags given and score given)
restInspections_byDate = restInspections_score.merge(restInspections_critFlag)

# group by each restaurant and take the average score for each restaurant for all inspections in 2022
restInspections_byCAMIS = restInspections_byDate.groupby(by='CAMIS').mean().reset_index()

# take only the most recent inspection for each unique restaurant so we only have one row per restaurant (this data currently only includes relevant features like zipcode and cusinie description)
restInspections.drop_duplicates(subset=['CAMIS'], keep='first', inplace=True)

# merge the dataset containing the calculated average score and critical flags for each restaurant back with the zipcode and cusinie description
restInspections = restInspections.merge(restInspections_byCAMIS, on='CAMIS')


In [17]:
# drop irrelevant columns
restInspections = restInspections.drop(['DBA','BUILDING', 'STREET', 'PHONE', 'GRADE DATE', 'GRADE', 'ACTION_x', 'CRITICAL FLAG_x' , 'SCORE_x',
                                        'INSPECTION TYPE', 'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CAMIS', 'Latitude', 'Longitude', 
                                        'RECORD DATE','Community Board', 'Council District', 'Census Tract', 
                                        'BIN', 'BBL','NTA', 'Location Point1'], axis=1)

restInspections.rename(columns={'SCORE_y': 'SCORE', 'CRITICAL FLAG_y': 'CRITICAL FLAG', 'ACTION_y': 'ACTION'}, inplace=True)


In [18]:
# one-hot encode cuisine description
processed_restInspections = pd.get_dummies(restInspections, columns = ['CUISINE DESCRIPTION'])

In [41]:
# groupby zip code and sum together total score, total critical flags, total closures, and totals for each cuisine by zipcode 
processed_restInspections_sum = processed_restInspections.groupby(by='ZIPCODE').sum()

# groupby zip code and take average score and average critical flags by zipcode
processed_restInspections_avg = processed_restInspections.groupby(by='ZIPCODE').mean()[['SCORE', 'CRITICAL FLAG']]
processed_restInspections_avg.rename(columns={'SCORE': 'AVG SCORE', 'CRITICAL FLAG': 'AVG FLAGS'}, inplace=True)

# join average score and critical flag data with other dataframe
processed_restInspections_df = processed_restInspections_sum.join(processed_restInspections_avg).reset_index()


### Open Restaurant Application Dataset Data Cleaning

In [20]:
# convert date data to datetime
openRest['Time of Submission'] = pd.to_datetime(openRest['Time of Submission'])

# drop rows that don't have any lat / long
openRest.dropna(subset=['Latitude'], inplace=True)

# keep only relevant columns
openRest = openRest[['Postcode', 'Food Service Establishment Permit #', 'Sidewalk Dimensions (Area)', 
                     'Roadway Dimensions (Area)', 'Approved for Sidewalk Seating', 'Approved for Roadway Seating', 
                     'Qualify Alcohol', 'Latitude', 'Longitude', 'Time of Submission']]

In [21]:
processed_openRest = openRest.copy()

# change yes/no to 1/0 for three categorical columns
processed_openRest['Approved for Sidewalk Seating'] = processed_openRest['Approved for Sidewalk Seating'].map(dict(yes=1, no=0))
processed_openRest['Approved for Roadway Seating'] = processed_openRest['Approved for Roadway Seating'].map(dict(yes=1, no=0))
processed_openRest['Qualify Alcohol'] = processed_openRest['Qualify Alcohol'].map(dict(yes=1, no=0))

In [22]:
# match open restaurant applications with restaurants that were active (had inspections) in 2022

# subset open restaurant applications for restaurants that have valid food service establishment permit #
processed_openRest.dropna(subset=['Food Service Establishment Permit #'], inplace=True)
processed_openRest = processed_openRest[processed_openRest['Food Service Establishment Permit #'].str.isnumeric() == True]
processed_openRest['Food Service Establishment Permit #'] = processed_openRest['Food Service Establishment Permit #'].astype('int64')

# get list of unique restaurant #s from inspection dataset (these are the restaurants that were active during 2022) & process data for merge
restList = pd.read_csv('raw_data\\DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
restList = restList[['CAMIS']].drop_duplicates()
restList['CAMIS'] = restList['CAMIS'].astype(int)

# merge datasets together to keep restaurants that were operating in 2022
processed_curr_openRest = restList.merge(processed_openRest, left_on='CAMIS', right_on='Food Service Establishment Permit #', how='inner')

In [23]:
# drop irrelevant columns
processed_curr_openRest = processed_curr_openRest.drop(['CAMIS', 'Food Service Establishment Permit #', 'Latitude', 'Longitude'], axis=1)

In [29]:
# groupby zipcode to get the number of open restaurants and the total seating area for each zipcode
processed_curr_openRest_df = processed_curr_openRest.groupby(by='Postcode').sum().reset_index()

### Rat Complaint Dataset Data Cleaning

In [25]:
# convert complaint date to datetime
rats_df['created_date'] = pd.to_datetime(rats_df['created_date'])

# keep only incident zipcode and date of complaint
rats_df = rats_df[['created_date', 'incident_zip']]

# filter out for rat complaints only in 2022
rats_2022_df = rats_df[(rats_df['created_date']>'2022') & (rats_df['created_date']<'2023')]

rats_2022_df.rename(columns={'created_date': 'RAT COUNT'}, inplace=True)
rats_2022_df = rats_2022_df.groupby(by='incident_zip').count().reset_index()


C:\Users\abonc\AppData\Local\Temp\ipykernel_38132\3697131064.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rats_2022_df.rename(columns={'created_date': 'RAT COUNT'}, inplace=True)


### Shapefile with NYC zip code polygons for plotting

In [26]:
# keep relevant columns
zipcodes = zipcodes[['ZIPCODE', 'POPULATION', 'AREA', 'geometry']]

# change zipcode to interger for merge
zipcodes['ZIPCODE'] = zipcodes['ZIPCODE'].astype(int)

### Merge Datasets together

In [43]:
# merge together processes restautant inspection, open restaurant application, and rat complaint datasets
features_df = processed_restInspections_df.merge(processed_curr_openRest_df,left_on=['ZIPCODE'], right_on=['Postcode'])
features_df = rats_2022_df.merge(features_df, left_on=['incident_zip'], right_on=['ZIPCODE']).drop(['incident_zip'], axis=1)

# merge geodataframe into processed dataset
full_df = zipcodes.merge(features_df)

In [44]:
full_df.head()

,ZIPCODE,POPULATION,AREA,geometry,RAT COUNT,SCORE,CRITICAL FLAG,ACTION,CUISINE DESCRIPTION_Afghan,CUISINE DESCRIPTION_African,...,CUISINE DESCRIPTION_Vegan,CUISINE DESCRIPTION_Vegetarian,AVG SCORE,AVG FLAGS,Postcode,Sidewalk Dimensions (Area),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol
0,11213,62426.0,2.963100e+07,"POLYGON ((1001613.713 186926.440, 1002314.243 ...",384,810.5,100.5,0.0,0,1,...,0,1,10.952703,1.358108,11213,4751.0,1848.0,14,9,8
1,11212,83866.0,4.197210e+07,"POLYGON ((1011174.276 183696.338, 1011373.584 ...",165,855.0,99.0,0.0,0,0,...,0,0,10.426829,1.207317,11212,528.0,688.0,6,6,4
2,11225,56527.0,2.369863e+07,"POLYGON ((995908.365 183617.613, 996522.848 18...",364,1054.5,127.5,0.0,0,0,...,0,2,11.461957,1.385870,11225,8963.0,4774.0,32,20,28
3,11218,72280.0,3.686880e+07,"POLYGON ((991997.113 176307.496, 992042.798 17...",315,1163.5,141.0,0.0,1,0,...,0,0,11.519802,1.396040,11218,16750.0,5500.0,41,19,22
4,11226,106132.0,3.940860e+07,"POLYGON ((994821.473 177865.718, 994891.817 17...",627,1677.0,185.5,0.0,0,2,...,1,4,10.889610,1.204545,11226,9653.0,3624.0,38,25,19


### Create CSV

In [ ]:
from pathlib import Path  
filepath = Path('processed_data/feature_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
full_df.to_csv(filepath)  